# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

In [1]:
'17'

'17'

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('bank-additional-full.csv', sep = ';')

In [6]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [8]:
#Several *categorical features* contain the string `"unknown"` to indicate missing data. These include: `job`, `marital`, `education`, `default`, `housing`, and `loan`. While technically not nulls (NaN), they should be treated as missing values and handled accordingly during data preprocessing.

# Identifying all the unknowns
for col in df.select_dtypes(include='object'):
    if 'unknown' in df[col].unique():
        print(f"{col}: {df[col].value_counts()['unknown']} unknowns")
        
#The `duration` feature, although numeric and highly predictive, should be removed from the modeling pipeline for a realistic prediction model. This is because it is only known **after** a client has been contacted and thus cannot be used for prior prediction.

job: 330 unknowns
marital: 80 unknowns
education: 1731 unknowns
default: 8597 unknowns
housing: 990 unknowns
loan: 990 unknowns


### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [70]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [11]:
'The objective of this project is to develop a predictive model that determines whether a customer will subscribe to a term deposit (target variable y) based on demographic, behavioral, and economic features. This model will help the bank’s marketing team identify high-potential customers before making a call, enabling more efficient and targeted telemarketing campaigns.'

'The objective of this project is to develop a predictive model that determines whether a customer will subscribe to a term deposit (target variable y) based on demographic, behavioral, and economic features. This model will help the bank’s marketing team identify high-potential customers before making a call, enabling more efficient and targeted telemarketing campaigns.'

In [12]:
'The goal is to increase customer conversion rates while minimizing operational costs and customer disturbance from irrelevant calls. Importantly, the variable duration will be excluded from modeling since it reflects information that would only be known after the customer has already been contacted.'

'The goal is to increase customer conversion rates while minimizing operational costs and customer disturbance from irrelevant calls. Importantly, the variable duration will be excluded from modeling since it reflects information that would only be known after the customer has already been contacted.'

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [56]:
import numpy as np
import pandas as pd

# Create a subset of the DataFrame with only the bank client features and the target
client_features = ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan']
target = 'y'

df_client = df[client_features + [target]].copy()

# Handle 'unknown' values by replacing them with NaN and dropping rows
df_client.replace('unknown', np.nan, inplace=True)
df_client.dropna(inplace=True)

#dropping meaningless features
columns_to_drop = [col for col in X.columns if 
                   col.startswith('duration') or 
                   col.startswith('contact_') or 
                   col.startswith('month_') or 
                   col.startswith('day_of_week_')]

X_filtered = X.drop(columns=columns_to_drop)


In [57]:
# Encode the target column
df_client['y'] = df_client['y'].map({'yes': 1, 'no': 0})

# Separate the features (X) from the target (y)
X = df_client.drop('y', axis=1)
y = df_client['y']

# Encode the categorical features
X_encoded = pd.get_dummies(X, drop_first=True)

# Replace X with the encoded features
X = X_encoded


In [58]:
# Output shape and sample
print("Encoded feature shape:", X.shape)
print("Sample encoded features:")
display(X.head())


Encoded feature shape: (30488, 22)
Sample encoded features:


,age,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,...,marital_single,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,default_yes,housing_yes,loan_yes
0,56,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,37,False,False,False,False,False,False,True,False,False,...,False,False,False,True,False,False,False,False,True,False
3,40,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
4,56,False,False,False,False,False,False,True,False,False,...,False,False,False,True,False,False,False,False,False,True
6,59,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False


### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [59]:
from sklearn.model_selection import train_test_split


In [60]:
test_size = 0.2
random_state = 42


In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)


### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [62]:
y.value_counts(normalize=True)


y
0    0.873426
1    0.126574
Name: proportion, dtype: float64

In [63]:
# Predict "no" (0) for every test observation

y_pred_baseline = [0] * len(y_test)  

In [64]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_baseline)


0.8697933748770088

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [65]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(max_iter=1000, class_weight='balanced')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)


0.6038045260741227

### Problem 9: Score the Model

What is the accuracy of your model?

In [66]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")


Accuracy: 0.6038
Precision: 0.1635
Recall: 0.4962
F1 Score: 0.2459
ROC AUC: 0.5980


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [67]:
import time
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score


In [68]:
# Define models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "k-NN": KNeighborsClassifier(),
    "SVM": SVC(probability=True, random_state=42)  # enable predict_proba for ROC AUC
}

results = []

# Loop through models
for name, model in models.items():
    start = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start

    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

    results.append({
        "Model": name,
        "Train Time": round(train_time, 3),
        "Train Accuracy": round(model.score(X_train, y_train), 4),
        "Test Accuracy": round(accuracy_score(y_test, y_pred), 4),
        "F1 Score": round(f1_score(y_test, y_pred), 4),
        "ROC AUC": round(roc_auc_score(y_test, y_proba), 4) if y_proba is not None else "N/A"
    })



In [36]:
# Create and show DataFrame
results_df = pd.DataFrame(results)
results_df 


,Model,Train Time,Train Accuracy,Test Accuracy,F1 Score,ROC AUC
0,Logistic Regression,0.480,0.8743,0.8698,0.0000,0.5955
1,Decision Tree,0.059,0.9017,0.8509,0.1368,0.5741
2,k-NN,0.008,0.8779,0.8595,0.1370,0.5778
3,SVM,66.144,0.8743,0.8698,0.0000,0.5581


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [39]:
# Drop duration if it exists (it's not usable before the call)
if 'duration' in X.columns:
    X = X.drop('duration', axis=1)


In [40]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier


In [38]:
dt_params = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10]
}
dt_model = DecisionTreeClassifier(class_weight='balanced', random_state=42)


In [41]:
grid_dt = GridSearchCV(dt_model, dt_params, scoring='f1', cv=5)
grid_dt.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=42),
             param_grid={'max_depth': [3, 5, 10, None],
                         'min_samples_split': [2, 5, 10]},
             scoring='f1')

In [42]:
print("Best Decision Tree params:", grid_dt.best_params_)


Best Decision Tree params: {'max_depth': 10, 'min_samples_split': 10}


In [43]:
from sklearn.metrics import f1_score, roc_auc_score

best_dt = grid_dt.best_estimator_
y_pred_dt = best_dt.predict(X_test)
print("F1:", f1_score(y_test, y_pred_dt))
print("ROC AUC:", roc_auc_score(y_test, best_dt.predict_proba(X_test)[:, 1]))



F1: 0.26254338908172925
ROC AUC: 0.6061325324549507


In [49]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [50]:
knn_params = {'n_neighbors': [3, 5, 7, 9]}
knn_model = KNeighborsClassifier()
grid_knn = GridSearchCV(knn_model, knn_params, scoring='f1', cv=5)
grid_knn.fit(X_train_scaled, y_train)



GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 7, 9]}, scoring='f1')

In [51]:
print("Best k-NN params:", grid_knn.best_params_)


Best k-NN params: {'n_neighbors': 3}


In [52]:
best_knn = grid_knn.best_estimator_
y_pred_knn = best_knn.predict(X_test_scaled)
print("F1:", f1_score(y_test, y_pred_knn))


F1: 0.136986301369863


In [53]:
print("Final F1 Comparison:")
print("Decision Tree:", round(f1_score(y_test, y_pred_dt), 4))
print("k-NN:", round(f1_score(y_test, y_pred_knn), 4))


Final F1 Comparison:
Decision Tree: 0.2625
k-NN: 0.137


In [71]:
# Create and show DataFrame
results_df = pd.DataFrame(results)
results_df 


,Model,Train Time,Train Accuracy,Test Accuracy,F1 Score,ROC AUC
0,Logistic Regression,0.797,0.8743,0.8698,0.0000,0.5955
1,Decision Tree,0.095,0.9017,0.8509,0.1368,0.5741
2,k-NN,0.016,0.8779,0.8595,0.1370,0.5778
3,SVM,109.021,0.8743,0.8698,0.0000,0.5581


##### Questions